In [1]:
# --- 1. INSTALLATION ---
print("⏳ Installing libraries... (This takes ~45 seconds)")
!pip install -q streamlit open_clip_torch transformers pandas

⏳ Installing libraries... (This takes ~45 seconds)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00


In [2]:
# @title 🧪 Option G: Turbocharged Balanced Training (Grad Accumulation + Scheduler)
import torch
import torch.nn as nn
import open_clip
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import pandas as pd
import random
import copy
import os
from torchvision import transforms

# --- 1. SETUP ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using Device: {device}")

# Load Base Model
print("🧠 Loading BioMedCLIP...")
model, _, preprocess_val = open_clip.create_model_and_transforms('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = open_clip.get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
model.to(device)

# --- 2. IMPROVED AUGMENTATION ---
# Changed CenterCrop to RandomResizedCrop for better generalization
train_transforms = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC), # Resize larger first
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)), # Random zoom (85% to 100%)
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(20), # Increased rotation slightly
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.1), # More robust lighting
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
])

# --- 3. BALANCED DATASET CLASS (Same as before) ---
class BalancedContrastiveDataset(Dataset):
    def __init__(self, hf_dataset, transform, tokenizer):
        self.tokenizer = tokenizer
        self.transform = transform

        self.label_map = {
            'melanoma': "High risk melanoma skin cancer",
            'melanocytic_nevi': "Benign melanocytic nevus mole",
            'basal_cell_carcinoma': "Basal cell carcinoma skin cancer",
            'actinic_keratoses': "Actinic keratosis pre-cancerous lesion",
            'benign_keratosis-like_lesions': "Benign keratosis-like lesion",
            'dermatofibroma': "Benign dermatofibroma skin lesion",
            'vascular_lesions': "Benign vascular skin lesion",
            # Fallbacks
            'mel': "High risk melanoma skin cancer",
            'nv': "Benign melanocytic nevus mole",
            'bcc': "Basal cell carcinoma skin cancer",
            'akiec': "Actinic keratosis pre-cancerous lesion",
            'bkl': "Benign keratosis-like lesion",
            'df': "Benign dermatofibroma skin lesion",
            'vasc': "Benign vascular skin lesion"
        }

        print("⚖️ Balancing Dataset...")
        data_by_class = {}
        for item in hf_dataset:
            label = str(item.get('dx', item.get('label', ''))).lower().strip()
            if label not in data_by_class: data_by_class[label] = []
            data_by_class[label].append(item)

        self.final_data = []
        for label, items in data_by_class.items():
            count = len(items)
            # Keeping the 1500 limit for Nevi
            if label in ['melanocytic_nevi', 'nv']:
                limit = 1500
                selected = random.sample(items, limit) if count > limit else items
                print(f"   🔻 Downsampling '{label}': {count} -> {len(selected)}")
            else:
                selected = items
                print(f"   ✅ Keeping all '{label}': {count}")
            self.final_data.extend(selected)

        print(f"   📊 Final Balanced Size: {len(self.final_data)} images")

        self.unique_labels = sorted(list(set(self.label_map.values())))
        self.text_to_idx = {text: i for i, text in enumerate(self.unique_labels)}

    def __len__(self): return len(self.final_data)

    def __getitem__(self, idx):
        item = self.final_data[idx]
        image = item['image'].convert("RGB")
        raw_label = str(item.get('dx', item.get('label', ''))).lower().strip()
        caption = self.label_map.get(raw_label, f"A dermoscopy image of {raw_label}")
        text_tokens = self.tokenizer([caption])[0]
        image_tensor = self.transform(image)
        class_idx = self.text_to_idx.get(caption, -1)
        return image_tensor, text_tokens, class_idx


✅ Using Device: cuda
🧠 Loading BioMedCLIP...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


open_clip_config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

open_clip_pytorch_model.bin:   0%|          | 0.00/784M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [3]:
# Load Data
print("📦 Loading & Splitting Data...")
full_train = load_dataset("marmal88/skin_cancer", split="train")
train_dataset = BalancedContrastiveDataset(full_train, train_transforms, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

📦 Loading & Splitting Data...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00005-7eed077f2f8e6d(…):   0%|          | 0.00/521M [00:00<?, ?B/s]

data/train-00001-of-00005-50ba64fd20294b(…):   0%|          | 0.00/525M [00:00<?, ?B/s]

data/train-00002-of-00005-36c02a25cbdd54(…):   0%|          | 0.00/527M [00:00<?, ?B/s]

...

data/test-00000-of-00001-61e7cf54bf274ae(…):   0%|          | 0.00/355M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9577 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2492 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1285 [00:00<?, ? examples/s]

⚖️ Balancing Dataset...
   ✅ Keeping all 'actinic_keratoses': 315
   ✅ Keeping all 'basal_cell_carcinoma': 487
   ✅ Keeping all 'benign_keratosis-like_lesions': 1048
   ✅ Keeping all 'dermatofibroma': 110
   🔻 Downsampling 'melanocytic_nevi': 6405 -> 1500
   ✅ Keeping all 'melanoma': 1076
   ✅ Keeping all 'vascular_lesions': 136
   📊 Final Balanced Size: 4672 images


In [4]:
val_data = load_dataset("marmal88/skin_cancer", split="validation")
val_dataset = BalancedContrastiveDataset(val_data, preprocess_val, tokenizer)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

⚖️ Balancing Dataset...
   ✅ Keeping all 'actinic_keratoses': 82
   ✅ Keeping all 'basal_cell_carcinoma': 124
   ✅ Keeping all 'benign_keratosis-like_lesions': 275
   ✅ Keeping all 'dermatofibroma': 30
   🔻 Downsampling 'melanocytic_nevi': 1666 -> 1500
   ✅ Keeping all 'melanoma': 280
   ✅ Keeping all 'vascular_lesions': 35
   📊 Final Balanced Size: 2326 images


In [5]:
test_data = load_dataset("marmal88/skin_cancer", split="test")
test_dataset = BalancedContrastiveDataset(test_data, preprocess_val, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

⚖️ Balancing Dataset...
   ✅ Keeping all 'actinic_keratoses': 42
   ✅ Keeping all 'basal_cell_carcinoma': 67
   ✅ Keeping all 'benign_keratosis-like_lesions': 142
   ✅ Keeping all 'dermatofibroma': 14
   🔻 Downsampling 'melanocytic_nevi': 858 -> 858
   ✅ Keeping all 'melanoma': 144
   ✅ Keeping all 'vascular_lesions': 18
   📊 Final Balanced Size: 1285 images


In [17]:
# --- 4. CONFIGURE MODEL (Higher Rank LoRA) ---
print("🔧 Configuring High-Capacity LoRA...")
def get_linear_layer_names(module):
    target_names = set()
    for name, module in module.named_modules():
        if isinstance(module, torch.nn.Linear):
            target_names.add(name.split('.')[-1])
    return list(target_names)

# Common target modules for LoRA in Vision Transformers
vision_targets = [
    "q_proj",
    "k_proj",
    "v_proj",
    "out_proj",
    "fc1",
    "fc2",
]
# CHANGE: Increased 'r' to 32 (was 16) for more learning capacity
config_vision = LoraConfig(r=32, lora_alpha=64, target_modules=vision_targets, lora_dropout=0.1, bias="none")
model.visual = get_peft_model(model.visual, config_vision)

for param in model.text.parameters():
    param.requires_grad = True

# --- 5. TRAINING LOOP (Scheduler + Gradient Accumulation) ---
# CHANGE: Increased LR slightly to 2e-5 because we have a scheduler now
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# CHANGE: Added Scheduler
num_epochs = 15
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

loss_img = torch.nn.CrossEntropyLoss()
loss_txt = torch.nn.CrossEntropyLoss()

# CHANGE: Gradient Accumulation Steps
# Effective Batch Size = 16 * 4 = 64
accumulation_steps = 4

best_val_acc = 0.0
best_model_state = None

# Evaluation Function
def evaluate(model, dataloader, unique_labels):
    model.eval()
    correct = 0; total = 0
    class_tokens = tokenizer(unique_labels).to(device)
    with torch.no_grad():
        class_embeddings = model.encode_text(class_tokens)
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)

        for images, _, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            img_feat = model.encode_image(images)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            similarity = (100.0 * img_feat @ class_embeddings.T).softmax(dim=-1)
            _, predicted = similarity.max(dim=1)
            mask = labels != -1
            correct += (predicted[mask] == labels[mask]).sum().item()
            total += mask.sum().item()
    return 100 * correct / total if total > 0 else 0

print(f"\n🚀 Starting Turbocharged Training (Effective Batch Size: {16*accumulation_steps})...")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad() # Initialize gradients

    # Progress bar
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for i, (images, texts, _) in enumerate(pbar):
        images, texts = images.to(device), texts.to(device)

        img_f, txt_f, logit_scale = model(images, texts)

        logits_img = logit_scale * img_f @ txt_f.T
        logits_txt = logits_img.T
        ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
        loss = (loss_img(logits_img, ground_truth) + loss_txt(logits_txt, ground_truth)) / 2

        # Normalize loss for accumulation
        loss = loss / accumulation_steps
        loss.backward()

        # Step optimizer only every 'accumulation_steps'
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps # Scale back up for reporting

    # Step Scheduler at end of epoch
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]

    # Validation
    val_acc = evaluate(model, val_loader, train_dataset.unique_labels)
    # print(f"   📉 Loss: {total_loss/len(train_loader):.4f} | 🏆 Val Acc: {val_acc:.2f}% ")
    print(f"   📉 Loss: {total_loss/len(train_loader):.4f} | 🏆 Val Acc: {val_acc:.2f}% | ⚡ LR: {current_lr:.2e}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"   💾 New High Score! ({val_acc:.2f}%) - Stashing Weights...")

🔧 Configuring High-Capacity LoRA...


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(



🚀 Starting Turbocharged Training (Effective Batch Size: 64)...


Epoch 1: 100%|██████████| 292/292 [06:08<00:00,  1.26s/it]


   📉 Loss: 2.0015 | 🏆 Val Acc: 72.91% | ⚡ LR: 2.97e-05
   💾 New High Score! (72.91%) - Stashing Weights...


...

   📉 Loss: 1.7588 | 🏆 Val Acc: 77.77% | ⚡ LR: 1.30e-06


Epoch 14: 100%|██████████| 292/292 [06:08<00:00,  1.26s/it]


   📉 Loss: 1.7494 | 🏆 Val Acc: 79.02% | ⚡ LR: 3.28e-07


Epoch 15: 100%|██████████| 292/292 [06:08<00:00,  1.26s/it]


   📉 Loss: 1.7530 | 🏆 Val Acc: 78.93% | ⚡ LR: 0.00e+00


In [18]:
# --- 6. FINAL SAVE ---
print("\n🏁 Training Complete. Restoring Best Weights...")
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    torch.save({'model_state_dict': best_model_state}, "/content/drive/MyDrive/Colab Notebooks/Computer Vision/Multimodal Medical/biomedclip_balanced_best3.pt")
    print(f"✅ Saved Best Model (Acc: {best_val_acc:.2f}%) as 'biomedclip_balanced_best.pt'")
else:
    print("⚠️ No improvement found.")

# Final Test
print("📝 Running Final Test...")
test_acc = evaluate(model, test_loader, train_dataset.unique_labels)
print(f"🎉 Final Test Set Accuracy: {test_acc:.2f}%")


🏁 Training Complete. Restoring Best Weights...
✅ Saved Best Model (Acc: 79.49%) as 'biomedclip_balanced_best.pt'
📝 Running Final Test...
🎉 Final Test Set Accuracy: 81.71%


In [16]:
from google.colab import files

files.download('/content/drive/MyDrive/Colab Notebooks/Computer Vision/Multimodal Medical/biomedclip_balanced_best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>